In [2]:
] activate "/Users/anarres/Documents/projects/BioXP"

 Activating environment at `~/Documents/projects/BioXP/Project.toml`


Some visual sugar

In [3]:
# ] add ProgressMeter

In [4]:
using BioXP
using ProgressMeter
using Random
using JSON
# using BenchmarkTools

### Inputs

In [4]:
## Organism JSONs from ecg (JGI)
input_dir = "data/input/rids/"

## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
rstructs_path = "data/input/rstructs/master_from_redges-og-submission.json"

## User defined seeds/targets
sid_sets_dir = "data/input/sid_randomizations/"
targets_path = "data/input/seeds/seeds.json"
tid_name = "targets_Freilich09";

### Path to write to

In [5]:
write_mdir = "data/output/minimal_seed_sets/";

if !ispath(write_mdir)
    mkpath(write_mdir)
end

Double check number of accessible threads

In [18]:
Threads.nthreads()

10

### Run minimal seed set expansions

Remember here that the threading is internal to the function

In [7]:
tids = readkeyedids(targets_path)[tid_name]
rstructs = readmaster(rstructs_path)

for path in readdir(input_dir)
    
    org_dir = joinpath(input_dir,path)
    
    if isdir(org_dir)
        
        p = Progress(length(readdir(org_dir)),desc="$path")
        
        for (i,fname) in collect(enumerate(readdir(org_dir)))
            if endswith(fname,".json")
    #             Random.seed!(seedoffset+i)
                rids = readids(joinpath(org_dir,fname))
                sid_sets_path = joinpath(sid_sets_dir,basename(org_dir),fname)
                sid_sets = convert(Vector{IDs},JSON.parsefile(sid_sets_path))
                write_path = joinpath(write_mdir,basename(org_dir),splitext(basename(fname))[1])
                
                if !ispath(joinpath(write_path))
                    mkpath(joinpath(write_path))
                end

                find_minimal_seed_set(rstructs,
                    rids,
                    sid_sets,
                    tids,
                    write_path)

#                 open(write_path,"w") do f
#                     JSON.print(f, random_seed_sets, 2) #indent=2
#                 end

                next!(p)
            end
        end
    end
end



bacteria100%|███████████████████████████████████████████| Time: 1 days, 20:02:55


### Format output

_I should format this for multithreading if I rerun it, because it takes awhile_

In [16]:
simple_output_dir = "data/output/minimal_seed_sets/"
for path1 in readdir(simple_output_dir)
    domain_dir = joinpath(simple_output_dir,path1)
    if isdir(domain_dir) &  !startswith(path1,".") 
        p = Progress(length(readdir(domain_dir)),desc="$path1")
        for path2 in readdir(domain_dir)
            org_dir = joinpath(domain_dir,path2)
            if isdir(org_dir) & !startswith(path2,".") 
                formatbioxpoutput(org_dir)
            end
            next!(p)
        end
    end
end

archaea100%|████████████████████████████████████████████| Time: 0:01:02
bacteria100%|███████████████████████████████████████████| Time: 0:19:10
